# Linear Regression Using PyTorch Built-ins

Begin by importing `torch.nn`

In [15]:
import torch.nn as nn
import numpy as np

In [4]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')
tensor_inputs = torch.from_numpy(inputs)
tensor_targets = torch.from_numpy(targets)         

In [5]:
tensor_inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

Now using `TensorDataset` we will access a small section of the data. The `TensorDataset` retuns a tuple in which first element contains input variables for selected rows and second contains the targets.
`DataLoader` will also be used to seperate training data into multiple batches.

In [7]:
from torch.utils.data import TensorDataset, DataLoader

In [10]:
train_ds = TensorDataset(tensor_inputs, tensor_targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [11]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [13]:
for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 69.,  96.,  70.],
        [ 69.,  96.,  70.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]])
tensor([[103., 119.],
        [103., 119.],
        [103., 119.],
        [ 22.,  37.],
        [ 81., 101.]])


Now using `nn.Linear` initialize weights and baises automatically

In [16]:
model = nn.Linear(3,2) #Here we are giving 3 inputs and expecting 2 outputs so we pass (3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.4222, -0.3010, -0.1563],
        [-0.0145,  0.0622, -0.3724]], requires_grad=True)
Parameter containing:
tensor([ 0.5709, -0.0041], requires_grad=True)


Now, the `.parameters` method returns list containing all weights and biases present in the model.

In [17]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4222, -0.3010, -0.1563],
         [-0.0145,  0.0622, -0.3724]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5709, -0.0041], requires_grad=True)]

Now we use `model` direclty for prediction. Also we use built in Loss function instead defining or own.

In [19]:
predictions = model(tensor_inputs)
print(predictions)

tensor([[  4.5070, -12.9110],
        [  2.5043, -19.6872],
        [-12.0934, -14.5318],
        [ 24.9138, -12.5914],
        [-10.1307, -21.1042],
        [  4.5070, -12.9110],
        [  2.5043, -19.6872],
        [-12.0934, -14.5318],
        [ 24.9138, -12.5914],
        [-10.1307, -21.1042],
        [  4.5070, -12.9110],
        [  2.5043, -19.6872],
        [-12.0934, -14.5318],
        [ 24.9138, -12.5914],
        [-10.1307, -21.1042]], grad_fn=<AddmmBackward>)


In [20]:
import torch.nn.functional as F

The `nn.functional` package contains loss functions and several other utilities.

In [21]:
loss_fn = F.mse_loss
loss = loss_fn(model(tensor_inputs),tensor_targets)
print(loss)

tensor(10409.9385, grad_fn=<MseLossBackward>)


Now we use an Optimizer `optim.SGD` Stochastic Gradient Descent to manipulate the weights and biases.

In [22]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Now the model is trained.

In [23]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Now we fit the data to the model

In [24]:
fit(1000, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 434.1135
Epoch [20/1000], Loss: 464.6451
Epoch [30/1000], Loss: 393.4149
Epoch [40/1000], Loss: 330.8832
Epoch [50/1000], Loss: 30.3313
Epoch [60/1000], Loss: 98.9102
Epoch [70/1000], Loss: 16.9290
Epoch [80/1000], Loss: 47.8669
Epoch [90/1000], Loss: 73.8317
Epoch [100/1000], Loss: 69.6348
Epoch [110/1000], Loss: 24.5665
Epoch [120/1000], Loss: 28.8273
Epoch [130/1000], Loss: 35.0946
Epoch [140/1000], Loss: 46.0903
Epoch [150/1000], Loss: 23.6097
Epoch [160/1000], Loss: 14.1558
Epoch [170/1000], Loss: 23.4076
Epoch [180/1000], Loss: 7.9011
Epoch [190/1000], Loss: 24.7366
Epoch [200/1000], Loss: 19.3902
Epoch [210/1000], Loss: 5.6412
Epoch [220/1000], Loss: 25.0803
Epoch [230/1000], Loss: 16.0434
Epoch [240/1000], Loss: 23.2031
Epoch [250/1000], Loss: 29.3071
Epoch [260/1000], Loss: 11.3857
Epoch [270/1000], Loss: 13.9427
Epoch [280/1000], Loss: 11.7649
Epoch [290/1000], Loss: 3.1106
Epoch [300/1000], Loss: 15.1587
Epoch [310/1000], Loss: 6.1529
Epoch [320/1000],